In [69]:
# Importing libraries
import sympy as smp
import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit

from matplotlib.animation import FuncAnimation

from IPython.display import HTML
import matplotlib as mpl


mpl.rcParams['animation.embed_limit'] = 50

plt.style.use('dark_background')

In [70]:
#Defining variabels and constants
t=smp.symbols('t', real='true',positive='true')
m1 , m2 , l1 , l2 , g = smp.symbols('m_1 m_2 l_1 l_2 g' , real='true' , positive='true')

In [71]:
#Defining the angles
theta1 = smp.Function('theta_1 ')(t)
theta2 = smp.Function('theta_2')(t)

In [72]:
#Defining the cartesian coordinates
x1=l1*smp.sin(theta1)
y1=-l1*smp.cos(theta1)

x2=x1+l2*smp.sin(theta2)
y2=y1-l2*smp.cos(theta2)

In [73]:
#Computing the velocities in cartesian coordinates
vx1=smp.diff(x1,t)
vy1=smp.diff(y1,t)

vx2=smp.diff(x2,t)
vy2=smp.diff(y2,t)

In [74]:
#Computing the energies
T1=smp.Rational(1,2) * m1 * (vx1**2 + vy1**2).simplify()
T2=smp.Rational(1,2) * m2 * (vx2**2 + vy2**2).simplify()


U1=m1*g*y1.simplify()
U2=m2*g*y2.simplify()


T=T1+T2.simplify()
U=U1+U2.simplify()

In [75]:
#The Lagrangian
L=T-U.simplify()

In [76]:
#Solving Euler-Lagrange equations for the 2 angles
dld1=smp.diff(L,theta1)
ddt_dld1=smp.diff(smp.diff(L,smp.diff(theta1,t)),t)

E1=dld1-ddt_dld1.simplify()


dld2=smp.diff(L,theta2)
ddt_dld2=smp.diff(smp.diff(L,smp.diff(theta2,t)),t)

E2=dld2-ddt_dld2.simplify()

In [77]:
#Solving each DE for the acceleration
sols = smp.solve([E1,E2],[smp.diff(theta1,t,2),smp.diff(theta2,t,2)])

In [92]:
#Linearizing the equation of motion by assuming small amplitudes and applying the small angle approximation
Linearized_sols = {key: val.subs({
    smp.sin(theta1): theta1,
    smp.cos(theta1): 1,
    smp.sin(theta2): theta2,
    smp.cos(theta2): 1,
    smp.sin(theta2 - theta1): theta2 - theta1,
    smp.cos(theta2 - theta1): 1,
    smp.sin(theta1 - theta2): theta1 - theta2,
    smp.cos(theta1 - theta2): 1,
    smp.sin(theta2 + theta1): theta2 + theta1,
    smp.cos(theta2 + theta1): 1,
    smp.sin(theta1 + theta2): theta1 + theta2,
    smp.cos(theta1 + theta2): 1
}).simplify() for key, val in sols.items()}


In [93]:
#Defining new names for the angles and angular velocities for convenience
u1, u2 , w1 , w2 = smp.symbols('u1 u2 w1 w2')

#Substituting the new names in the DEs
f_expr=Linearized_sols[smp.diff(theta1,t,2)].subs({
    smp.diff(theta1,t): 0,
    smp.diff(theta2,t): 0,
    theta1:u1,
    theta2:u2
})


q_expr=Linearized_sols[smp.diff(theta2,t,2)].subs({
    smp.diff(theta1,t): 0,
    smp.diff(theta2,t): 0,
    theta1:u1,
    theta2:u2
})

In [94]:
#Substituting values for the constants (all 1s for simplicity)
m_1 , m_2 , l_1 , l_2 , grav= 1, 1, 1, 1, 1
params = {m1: m_1, m2: m_2, l1: l_1, l2: l_2, g: grav}


f_expr_num = f_expr.subs(params)
q_expr_num = q_expr.subs(params)


#Converting the symolic expression to a numerical one
f = smp.lambdify((u1, u2), f_expr_num, "numpy")
q = smp.lambdify((u1, u2), q_expr_num, "numpy")

In [95]:
# Right-hand side of the ODE system
def rhs(t,x):
    u1 , u2 , w1 , w2 = x
    du1_dt=w1
    du2_dt=w2
    dw1_dt=f(u1 , u2 )
    dw2_dt=q(u1 , u2 )
    return [du1_dt , du2_dt , dw1_dt , dw2_dt]

In [97]:
#Constructing the coefficient matrix
A = smp.Matrix([[smp.diff(f_expr_num, u1), smp.diff(f_expr_num, u2)],
               [smp.diff(q_expr_num, u1), smp.diff(q_expr_num, u2)]])
A_num = np.array(A).astype(np.float64)

#Finding the eigenvalues and eigenvectors associated with it to find the normal modes
eigvals, eigvecs = np.linalg.eig(A_num)
omega_sq = -eigvals
omega = np.sqrt(omega_sq)

In [83]:
#Settng Initial conditions to match the eigenvectors (normal modes)
u10=0.01*(eigvecs[0][0])
u20=0.01*(eigvecs[1][0])

w10=0
w20=0

x0=[u10 , u20 , w10 , w20]

tsp=(0,50)
t_eval=np.linspace(tsp[0],tsp[1],500)

In [84]:
#Solveing the DE
sol = solve_ivp(rhs, tsp, x0, t_eval=t_eval, method='DOP853',rtol=1e-9, atol=1e-9)

In [85]:
#Extracting the array of values from the solver
u1 = sol.y[0]
u2 = sol.y[1]
w1 = sol.y[2]
w2 = sol.y[3]

In [ ]:
#Plotting angular positions (angles) Vs. time
plt.plot(t_eval,u1,label='bob 1',color='orange')
plt.plot(t_eval,u2,label='bob 2')
plt.xlabel('Time')
plt.ylabel('Angular position')
plt.legend()
plt.grid()

In [87]:
#Converting to cartesian
x1=0.55*np.sin(u1)
y1=-0.55*np.cos(u1)

x2=x1+0.75*np.sin(u2)
y2=y1-0.75*np.cos(u2)

In [ ]:
#Animating the trajectory of the double pendulum

#Setting up the figure environment
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)


#Create line artists
line, = ax.plot([], [], 'o-', lw=2)
trace1, = ax.plot([], [], 'b-', lw=0.5)
trace2, = ax.plot([], [], 'r-', lw=0.5)


#Arrays for path of bobs
path1_x, path1_y = [], []
path2_x, path2_y = [], []



#Initialization function
def init():
    line.set_data([], [])
    trace1.set_data([], [])
    trace2.set_data([], [])
    return line, trace1, trace2

#Updating function
def update(frame):
    
    line.set_data([0, x1[frame], x2[frame]],
                  [0, y1[frame], y2[frame]])

    
    path1_x.append(x1[frame])
    path1_y.append(y1[frame])
    trace1.set_data(path1_x, path1_y)

    path2_x.append(x2[frame])
    path2_y.append(y2[frame])
    trace2.set_data(path2_x, path2_y)

    return line, trace1, trace2

#Animation
ani = FuncAnimation(fig, update, frames=len(u1),
                    init_func=init, blit=True, interval=30)
#Animation display using HTML5
HTML(ani.to_jshtml())

In [ ]:
#Fourier tranforming the trajectories to check the frequecny

dt = t_eval[1] - t_eval[0]
N = len(t_eval)

# Compute FFT
fft_u1 = np.abs(np.fft.rfft(u1))
fft_u2 = np.abs(np.fft.rfft(u2))
freqs = np.fft.rfftfreq(N, dt)


# Plot
plt.figure(figsize=(8,4))
plt.plot(freqs, fft_u1, label='u1 (bob 1)')
plt.plot(freqs, fft_u2, label='u2 (bob 2)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.title('Fourier Transform of Linearized Double Pendulum Mode')
plt.grid(True)
plt.legend()

In [ ]:
# Find index of the peak for u1
idx_peak_u1 = np.argmax(fft_u1)
freq_peak_u1 = freqs[idx_peak_u1]        # in Hz
omega_peak_u1 = 2 * np.pi * freq_peak_u1    # in rad/s
amplitude_u1 = 2 * fft_u1[idx_peak_u1] / N  # factor 2/N for single-sided FFT

print("Bob 1:")
print("Peak frequency (Hz):", freq_peak_u1)
print("Angular frequency (rad/s):", omega_peak_u1)
print("Amplitude:", amplitude_u1)


# Same for u2
idx_peak_u2 = np.argmax(fft_u2)
freq_peak_u2 = freqs[idx_peak_u2]        # in Hz
omega_peak_u2 = 2 * np.pi * freq_peak_u2    # in rad/s
amplitude_u2 = 2 * fft_u2[idx_peak_u2] / N  # factor 2/N for single-sided FFT

print("\nBob 2:")
print("Peak frequency (Hz):", freq_peak_u2)
print("Angular frequency (rad/s):", omega_peak_u2)
print("Amplitude:", amplitude_u2)

# Amplitude ratio
print("\nAmplitude ratio (u1/u2):", amplitude_u1 / amplitude_u2)